1. Сделать SDF-file из CSV таблиц 
2. Отфильтровать структуры используя этот инструмент. 
3. Установить sklearn , CatBoost , XGBoost , LightGBM  
4. Файл SCAMS.csv использовать в 10-fold cross-validation (тут эти части будут называться train
and validation). Использовать всю выборку test_DLS.csv в качестве тестовой.
5. Обучить хотя бы одну модель из каждого пакета выше. Посчитать ROC-AUC lower confidence
interval, ROC-AUC, ROC-AUC upper confidence interval, MCC, Accuracy, F1 в ходе 10-fold crossvalidation
и выбрать лучшую модель для каждого пакета. Сохранить все значения и построить
для violin-plot (группировка по метрикам, затем название каждой модели) по результатам 10-
fold cross-validation. Затем построить radar-chart для лучших моделей.

### Установить sklearn +, CatBoost +, XGBoost +, LightGBM +

In [ ]:
pip install scikit-learn

In [ ]:
pip install xgboost 


Solving environment: ...working... done

## Package Plan ##

  environment location: C:\More_Program_Files\Anaconda\envs\my-rdkit-env

  added / updated specs:
    - xgboost


The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    conda-forge::ca-certificates-2021.10.~ --> conda-forge/label/cf202003::ca-certificates-2019.11.28-hecc5488_0


Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install  catboost 

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\More_Program_Files\Anaconda\envs\my-rdkit-env

  added / updated specs:
    - catboost


The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    conda-forge/label/cf202003::ca-certif~ --> conda-forge/label/cf201901::ca-certificates-2018.11.29-ha4d7672_0


Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install  lightgbm

Solving environment: ...working... done


Note: you may need to restart the kernel to use updated packages.
## Package Plan ##

  environment location: C:\More_Program_Files\Anaconda\envs\my-rdkit-env

  added / updated specs:
    - lightgbm


The following packages will be UPDATED:

  ca-certificates    conda-forge/label/cf201901::ca-certif~ --> conda-forge::ca-certificates-2021.10.8-h5b45459_0


Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done


In [ ]:
conda install -c richlewis scikit-chem

In [ ]:
# Сделать SDF-file из CSV таблиц

In [2]:
import rdkit as rdkit

In [8]:
import pandas as pd
import numpy as np
import os
import rdkit
from rdkit import rdBase
from rdkit import Chem
from rdkit import RDConfig

from rdkit.Chem import rdRGroupDecomposition
from rdkit.Chem import PandasTools

In [ ]:
scams = pd.read_csv('SCAMS.csv')
PandasTools.AddMoleculeColumnToFrame(scams, 'Smiles String','Molecule')
scams

In [55]:
# сделать sdf файл
# все столбци из исходного csv сохраняются как тэги, это надо ?
PandasTools.WriteSDF(scams, 'scams_out.sdf', molColName='Molecule', properties=list(scams.columns))

In [10]:
# исключить спорные структуры полученные при ChemSar
scams.drop(labels=[254, 451, 470, 472, 474, 516, 97, 459], axis=0,  inplace=True, errors='raise')

In [ ]:
#  убрать лишнюю нумерацию
del scams['Unnamed: 0']
# перенумеровать ряды
scams_filt = scams.reset_index(drop=True, inplace=False)

In [4]:
# прочитать  тестовый файл

test_set = pd.read_csv('test_DLS.csv') # прочитать тестовую выборку
PandasTools.AddMoleculeColumnToFrame(test_set, 'Smiles String','Molecule', includeFingerprints=False)

In [10]:
# agg? начения сделать int

scams_filt['agg?'] = scams_filt['agg?'].apply(lambda x: int(x))
test_set['agg?'] = test_set['agg?'].apply(lambda x: int(x))

In [13]:
#  схранить как sdf
PandasTools.WriteSDF(test_set, 'test_set.sdf', molColName='Molecule', properties=list(test_set.columns))
PandasTools.WriteSDF(scams_filt, 'scams_filt.sdf', molColName='Molecule', properties=list(scams_filt.columns))

# test_y = test_df['agg?']
# молекулы - 'Smiles String', 'Molecule'